In [2]:
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install prettytable
!pip install pandas


  Using cached SQLAlchemy-1.3.9-cp312-cp312-win_amd64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.40
    Uninstalling SQLAlchemy-2.0.40:
      Successfully uninstalled SQLAlchemy-2.0.40


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aext-project-filebrowser-server 4.1.0 requires sqlalchemy>=2.0.29, but you have sqlalchemy 1.3.9 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.


  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl (2.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [4]:
import sqlite3
import pandas as pd

# Connect to SQLite
con = sqlite3.connect("my_data1.db")
cur = con.cursor()


In [6]:
df = pd.read_csv('Spacex.csv')
df.head()


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [8]:
# Remove blank rows (optional clean up)
df = df[df['Date'].notna()]

# Save into the database
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False)


101

In [10]:
query = "SELECT DISTINCT \"Launch_Site\" FROM SPACEXTBL"
result = pd.read_sql(query, con)
result


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


In [12]:
query = """
SELECT *
FROM SPACEXTBL
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5
"""
result = pd.read_sql(query, con)
result


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [14]:
query = """
SELECT SUM("PAYLOAD_MASS__KG_") AS Total_Payload_Mass
FROM SPACEXTBL
WHERE "Customer" = 'NASA (CRS)'
"""
result = pd.read_sql(query, con)
result


,Total_Payload_Mass
0,45596


In [16]:
query = """
SELECT AVG("PAYLOAD_MASS__KG_") AS Average_Payload
FROM SPACEXTBL
WHERE "Booster_Version" = 'F9 v1.1'
"""
result = pd.read_sql(query, con)
result


,Average_Payload
0,2928.4


In [18]:
query = """
SELECT MIN(Date) AS First_Ground_Pad_Success
FROM SPACEXTBL
WHERE "Landing_Outcome" = 'Success (ground pad)'
"""
result = pd.read_sql(query, con)
result


,First_Ground_Pad_Success
0,2015-12-22


In [20]:
query = """
SELECT "Booster_Version"
FROM SPACEXTBL
WHERE "Landing_Outcome" = 'Success (drone ship)'
AND "PAYLOAD_MASS__KG_" > 4000
AND "PAYLOAD_MASS__KG_" < 6000
"""
result = pd.read_sql(query, con)
result


,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


In [22]:
query = """
SELECT "Mission_Outcome", COUNT(*) AS Outcome_Count
FROM SPACEXTBL
GROUP BY "Mission_Outcome"
"""
result = pd.read_sql(query, con)
result


,Mission_Outcome,Outcome_Count
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


In [24]:
query = """
SELECT "Booster_Version", "PAYLOAD_MASS__KG_"
FROM SPACEXTBL
WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTBL)
"""
result = pd.read_sql(query, con)
result


,Booster_Version,PAYLOAD_MASS__KG_
0,F9 B5 B1048.4,15600
1,F9 B5 B1049.4,15600
2,F9 B5 B1051.3,15600
3,F9 B5 B1056.4,15600
4,F9 B5 B1048.5,15600
5,F9 B5 B1051.4,15600
6,F9 B5 B1049.5,15600
7,F9 B5 B1060.2,15600
8,F9 B5 B1058.3,15600
9,F9 B5 B1051.6,15600


In [26]:
query = """
SELECT substr(Date, 6, 2) AS Month,
       "Landing_Outcome",
       "Booster_Version",
       "Launch_Site"
FROM SPACEXTBL
WHERE substr(Date, 1, 4) = '2015'
AND "Landing_Outcome" LIKE '%drone ship%'
"""
result = pd.read_sql(query, con)
result


,Month,Landing_Outcome,Booster_Version,Launch_Site
0,01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
2,06,Precluded (drone ship),F9 v1.1 B1018,CCAFS LC-40


In [28]:
query = """
SELECT "Landing_Outcome", COUNT(*) AS Outcome_Count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY Outcome_Count DESC
"""
result = pd.read_sql(query, con)
result


,Landing_Outcome,Outcome_Count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1
